In [58]:
import os
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

In [59]:
compare_folder = "/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Compare/2021-04-09_COVID"
#compare_folder = "/home/laura/ANALYSIS/MISC/Reinf_triple/compare"
pairwise_filename = [x for x in os.listdir(compare_folder) if '.snp.pairwise.tsv' in x][0]
pairwise_file = os.path.join(compare_folder, pairwise_filename)
pwdf = pd.read_csv(pairwise_file, sep="\t" , names=['sample_1', 'sample_2', 'dist'])

In [60]:
pwdf.head()

,sample_1,sample_2,dist
0,20461953,20461953,0
1,20461953,21086712-BIS,13
2,20461953,20490128,10
3,20461953,21148461,13
4,20461953,21119979,35


In [61]:
pwdf['sample_1'] = pwdf['sample_1'].astype(str)

In [62]:
pwdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7193124 entries, 0 to 7193123
Data columns (total 3 columns):
sample_1    object
sample_2    object
dist        int64
dtypes: int64(1), object(2)
memory usage: 164.6+ MB


In [63]:
samples = list(set(pwdf.sample_1.tolist()))

In [64]:
len(samples)

2682

In [65]:
samples_type = [type(x) for x in samples]
dict((x,samples_type.count(x)) for x in set(samples_type))

{str: 2682}

In [66]:
wgs = pd.read_csv('/media/NASII/Datos/ANALYSIS/WGS_sequenced/WGS_sequenced.csv')
#wgs = pd.read_csv('/home/laura/ANALYSIS/MISC/microreact/WGS_sequenced.csv')
wgs = wgs.rename(columns={'Muestra': 'id', 'Fecha': 'full_date', 'SEXO' : 'SEX__autocolor', 'EDAD': 'AGE__autocolor', 'Día':'day', 'Mes' : 'month', 'Año':'year', 'RUN':'RUN__autocolor'})

In [67]:
wgs.head(2)

,Apellidos,Nombre,NHC,CIPA,AGE__autocolor,SEX__autocolor,CENTRO SANITARIO,id,Sufijo,COV>30X,RUN__autocolor,DEPARTMENT__autocolor,patient_microreact,full_date,day,month,year,Extracción Muestra,GenN2_Ct,CT1_PCR1,Antígeno COVID
0,¡,FRANCISCO,2198204,NaN,71.122375,M,HOSPITAL GENERAL UNIVERSITARIO GREGORIO MARAÑÓN,20423238,NaN,3.63,RUN_10,NaN,NaN,28/10/2020,28.0,10.0,2020.0,EX. NASAL - FARÍNGEO,17.0,NaN,NaN
1,AAROUD,AHMIDOU,11031999,NaN,77.000000,M,HOSPITAL GENERAL UNIVERSITARIO GREGORIO MARAÑÓN,20097501,NaN,39.43,RUN_12,NaN,NaN,08/04/2020,8.0,4.0,2020.0,NaN,NaN,NaN,NaN


In [68]:
wgs.columns

Index(['Apellidos', 'Nombre', 'NHC', 'CIPA', 'AGE__autocolor',
       'SEX__autocolor', 'CENTRO SANITARIO', 'id', 'Sufijo', 'COV>30X',
       'RUN__autocolor', 'DEPARTMENT__autocolor', 'patient_microreact',
       'full_date', 'day', 'month', 'year', 'Extracción Muestra', 'GenN2_Ct',
       'CT1_PCR1', 'Antígeno COVID'],
      dtype='object')

In [69]:
wgs = wgs[['id', 'full_date', 'SEX__autocolor', 'AGE__autocolor', 'day', 'month', 'year', 'RUN__autocolor']]

In [70]:
wgs.head()

,id,full_date,SEX__autocolor,AGE__autocolor,day,month,year,RUN__autocolor
0,20423238,28/10/2020,M,71.122375,28.0,10.0,2020.0,RUN_10
1,20097501,08/04/2020,M,77.000000,8.0,4.0,2020.0,RUN_12
2,20395396,9/10/2020,M,80.210930,9.0,10.0,2020.0,RUN_9
3,20091284,2/4/2020,M,NaN,2.0,4.0,2020.0,RUN_1
4,20519706,30/12/2020,F,40.042522,30.0,12.0,2020.0,MICRO_17


In [71]:
wgs.shape

(2952, 8)

In [72]:
wgs['id'] = wgs['id'].astype(str)

In [73]:
samples_wgs = wgs.id.tolist()
#samples_wgs = [str(x) for x in samples_wgs]

In [74]:
WGS = set(samples_wgs)
PW = set(samples)

In [75]:
len(PW)

2682

In [76]:
missing_wgs = list(PW - WGS)
len(missing_wgs)

186

In [77]:
missing_wgs_type = [type(x) for x in missing_wgs]
dict((x,missing_wgs_type.count(x)) for x in set(missing_wgs_type))

{str: 186}

In [78]:
missing_pw = list(WGS - PW)
len(missing_pw)

252

In [79]:
all_missing = missing_pw + missing_wgs
len(all_missing)

438

In [80]:
wgs.shape

(2952, 8)

In [81]:
df = wgs[~wgs.id.isin(missing_pw)]

In [82]:
df.shape

(2662, 8)

In [83]:
df2 = pd.DataFrame(missing_wgs, columns=['id'])

In [84]:
df2['id'] = df2['id'].astype(str)

In [85]:
df2.head()

,id
0,20401363BIS
1,20464767BIS
2,20455752BIS
3,20190147Micro
4,20458985BIS


In [86]:
df.head()

,id,full_date,SEX__autocolor,AGE__autocolor,day,month,year,RUN__autocolor
3,20091284,2/4/2020,M,NaN,2.0,4.0,2020.0,RUN_1
5,20289362,25/8/2020,F,39.694927,25.0,8.0,2020.0,MICRO_17
6,20432945,4/11/2020,F,28.247288,4.0,11.0,2020.0,RUN_11
7,21035911,23/1/2021,F,49.521111,23.0,1.0,2021.0,RUN_19
8,21020388,15/01/2021,M,76.000000,15.0,1.0,2021.0,RUN_17


In [87]:
final_df = df.append(df2).reset_index(drop=True)
final_df = final_df[ ['id', 'full_date', 'SEX__autocolor', 'AGE__autocolor', 'day', 'month', 'year', 'RUN__autocolor'] ]


/home/laura/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [88]:
final_df.head()

,id,full_date,SEX__autocolor,AGE__autocolor,day,month,year,RUN__autocolor
0,20091284,2/4/2020,M,NaN,2.0,4.0,2020.0,RUN_1
1,20289362,25/8/2020,F,39.694927,25.0,8.0,2020.0,MICRO_17
2,20432945,4/11/2020,F,28.247288,4.0,11.0,2020.0,RUN_11
3,21035911,23/1/2021,F,49.521111,23.0,1.0,2021.0,RUN_19
4,21020388,15/01/2021,M,76.000000,15.0,1.0,2021.0,RUN_17


In [89]:
metadata = pd.read_csv('/home/laura/ANALYSIS/MISC/microreact/wgs_covid_metadata_REFORMATED.csv', encoding='latin1')
metadata['id'] = metadata['id'].astype(str)

In [90]:
metadata.tail()

,id,DEPARTMENT__autocolor,LABEL__autocolor
2209,21148572,DIVERSIDAD,DIVERSIDAD
2210,21148366,DIVERSIDAD,DIVERSIDAD
2211,21148425,DIVERSIDAD,DIVERSIDAD
2212,21148432,DIVERSIDAD,DIVERSIDAD
2213,21150553,DIVERSIDAD,DIVERSIDAD


In [91]:
final_df = final_df.fillna(0)
final_df.day = final_df.day.astype('Int64')
final_df.month = final_df.month.astype('Int64')
final_df.year = final_df.year.astype('Int64')
final_df.replace(0, np.nan, inplace=True)

In [92]:
final_df = final_df.merge(metadata, how='outer')

In [93]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3031 entries, 0 to 3030
Data columns (total 10 columns):
id                       3031 non-null object
full_date                2666 non-null object
SEX__autocolor           2469 non-null object
AGE__autocolor           2583 non-null float64
day                      2666 non-null Int64
month                    2666 non-null Int64
year                     2666 non-null Int64
RUN__autocolor           2590 non-null object
DEPARTMENT__autocolor    2376 non-null object
LABEL__autocolor         2376 non-null object
dtypes: Int64(3), float64(1), object(6)
memory usage: 269.4+ KB


In [94]:
final_df.tail()

,id,full_date,SEX__autocolor,AGE__autocolor,day,month,year,RUN__autocolor,DEPARTMENT__autocolor,LABEL__autocolor
3026,21148332,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAM (COMUNITARIO),CAM
3027,21148346,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DIVERSIDAD,DIVERSIDAD
3028,21145405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DIVERSIDAD,DIVERSIDAD
3029,21148572,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DIVERSIDAD,DIVERSIDAD
3030,21148366,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DIVERSIDAD,DIVERSIDAD


In [95]:
def extract_lineage(sample):
    pango_file = os.path.join('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Annotation/pangolin', str(sample) + ".lineage.csv")
    if os.path.exists(pango_file):
        df = pd.read_csv(pango_file)
        return df.lineage[0]
    else:
        return None

In [96]:
final_df['Lineage__autocolor'] = final_df.apply(lambda x: extract_lineage(x['id']), axis=1)

In [97]:
def extract_EU(sample):
    eu_file = os.path.join('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Annotation/user', str(sample) + ".tsv")
    if os.path.exists(eu_file):
        df = pd.read_csv(eu_file, sep="\t")
        if 'Nextrain_all' in df.columns:
            eulist = df.Nextrain_all.dropna().tolist()
            return (';').join([(x + "-" + str(eulist.count(x))) for x in set(eulist)])
        else:
            return None

In [98]:
final_df['Nextrain__autocolor'] = final_df.apply(lambda x: extract_EU(x['id']), axis=1)

/home/laura/anaconda3/lib/python3.7/site-packages/pandas/core/apply.py:186: DtypeWarning: Columns (20,21,22,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  return self.apply_standard()
/home/laura/anaconda3/lib/python3.7/site-packages/pandas/core/apply.py:186: DtypeWarning: Columns (22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  return self.apply_standard()
/home/laura/anaconda3/lib/python3.7/site-packages/pandas/core/apply.py:186: DtypeWarning: Columns (20,22,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  return self.apply_standard()
/home/laura/anaconda3/lib/python3.7/site-packages/pandas/core/apply.py:186: DtypeWarning: Columns (20,21,23) have mixed types. Specify dtype option on import or set low_memory=False.
  return self.apply_standard()


In [99]:
def extract_resistance(sample):
    res_file = os.path.join('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Annotation/user_aa', str(sample) + ".tsv")
    if os.path.exists(res_file):
        df = pd.read_csv(res_file, sep="\t")
        if 'resistance_covid' in df.columns:
            relist = df.resistance_covid.dropna().tolist()
            return (';').join([(x + "-" + str(relist.count(x))) for x in set(relist)])
        else:
            return None

In [100]:
final_df['Resistance__autocolor'] = final_df.apply(lambda x: extract_resistance(x['id']), axis=1)

/home/laura/anaconda3/lib/python3.7/site-packages/pandas/core/apply.py:186: DtypeWarning: Columns (26,27,29,30,31,32,33,34,36) have mixed types. Specify dtype option on import or set low_memory=False.
  return self.apply_standard()
/home/laura/anaconda3/lib/python3.7/site-packages/pandas/core/apply.py:186: DtypeWarning: Columns (26,29,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  return self.apply_standard()
/home/laura/anaconda3/lib/python3.7/site-packages/pandas/core/apply.py:186: DtypeWarning: Columns (6,26,27,28,29,30,31,33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  return self.apply_standard()
/home/laura/anaconda3/lib/python3.7/site-packages/pandas/core/apply.py:186: DtypeWarning: Columns (26,27,29,31,32,35) have mixed types. Specify dtype option on import or set low_memory=False.
  return self.apply_standard()
/home/laura/anaconda3/lib/python3.7/site-packages/pandas/core/apply.py:186: DtypeWarning: C

In [101]:
"""
def extract_variant(sample):
    res_file = os.path.join('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Annotation/user_aa', str(sample) + ".tsv")
    if os.path.exists(res_file):
        df = pd.read_csv(res_file, sep="\t")
        if 'Variants' in df.columns:
            df = df.drop_duplicates(subset=['Variants'], keep="first")
            relist = df.Variants.dropna().tolist()
            return (',').join([(x + "-" + str(relist.count(x))) for x in set(relist)])
        else:
            return None
"""

'\ndef extract_variant(sample):\n    res_file = os.path.join(\'/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Annotation/user_aa\', str(sample) + ".tsv")\n    if os.path.exists(res_file):\n        df = pd.read_csv(res_file, sep="\t")\n        if \'Variants\' in df.columns:\n            df = df.drop_duplicates(subset=[\'Variants\'], keep="first")\n            relist = df.Variants.dropna().tolist()\n            return (\',\').join([(x + "-" + str(relist.count(x))) for x in set(relist)])\n        else:\n            return None\n'

In [102]:
#final_df['VariantsAll__autocolor'] = final_df.apply(lambda x: extract_variant(x['id']), axis=1)

In [103]:
"""
def extract_variant_count(sample):
    res_file = os.path.join('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Annotation/user_aa', str(sample) + ".tsv")
    if os.path.exists(res_file):
        df = pd.read_csv(res_file, sep="\t")
        if 'Variants' in df.columns:
            df = df.drop_duplicates(subset=['Variants'], keep="first")
            relist = df.Variants.dropna().tolist()
            relist = [x.split(":")[1] for x in relist]
            return (',').join([(x + "-" + str(relist.count(x))) for x in set(relist)])
        else:
            return None
"""

'\ndef extract_variant_count(sample):\n    res_file = os.path.join(\'/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Annotation/user_aa\', str(sample) + ".tsv")\n    if os.path.exists(res_file):\n        df = pd.read_csv(res_file, sep="\t")\n        if \'Variants\' in df.columns:\n            df = df.drop_duplicates(subset=[\'Variants\'], keep="first")\n            relist = df.Variants.dropna().tolist()\n            relist = [x.split(":")[1] for x in relist]\n            return (\',\').join([(x + "-" + str(relist.count(x))) for x in set(relist)])\n        else:\n            return None\n'

In [104]:
#final_df['VariantsCount__autocolor'] = final_df.apply(lambda x: extract_variant_count(x['id']), axis=1)

In [105]:
def extract_variant_UK(sample): #Change header
    res_file = os.path.join('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Annotation/user_aa', str(sample) + ".tsv")
    if os.path.exists(res_file):
        df = pd.read_csv(res_file, sep="\t")
        if 'UKVariant' in df.columns: #Change header
            df = df.drop_duplicates(subset=['UKVariant'], keep="first") #Change header
            df = df[df.ALT_FREQ >= 0.4]
            relist = df.UKVariant.dropna().tolist() #Change header
            relist = [x.split(":")[1] for x in relist]
            return (',').join([(x + "-" + str(relist.count(x))) for x in set(relist)])
        else:
            return None

def extract_variant_D(sample): #Change header
    res_file = os.path.join('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Annotation/user_aa', str(sample) + ".tsv")
    if os.path.exists(res_file):
        df = pd.read_csv(res_file, sep="\t")
        if 'DVariant' in df.columns: #Change header
            df = df.drop_duplicates(subset=['DVariant'], keep="first") #Change header
            df = df[df.ALT_FREQ >= 0.4]
            relist = df.DVariant.dropna().tolist() #Change header
            relist = [x.split(":")[1] for x in relist]
            return (',').join([(x + "-" + str(relist.count(x))) for x in set(relist)])
        else:
            return None

def extract_variant_69del(sample): #Change header
    res_file = os.path.join('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Annotation/user_aa', str(sample) + ".tsv")
    if os.path.exists(res_file):
        df = pd.read_csv(res_file, sep="\t")
        if '69del' in df.columns: #Change header
            df = df.drop_duplicates(subset=['69del'], keep="first") #Change header
            df = df[df.ALT_FREQ >= 0.4]
            relist = df['69del'].dropna().tolist() #Change header
            relist = [x.split(":")[1] for x in relist]
            return (',').join([(x + "-" + str(relist.count(x))) for x in set(relist)])
        else:
            return None

def extract_variant_P1Variant(sample): #Change header
    res_file = os.path.join('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Annotation/user_aa', str(sample) + ".tsv")
    if os.path.exists(res_file):
        df = pd.read_csv(res_file, sep="\t")
        if 'P1Variant' in df.columns: #Change header
            df = df.drop_duplicates(subset=['P1Variant'], keep="first") #Change header
            df = df[df.ALT_FREQ >= 0.4]
            relist = df['P1Variant'].dropna().tolist() #Change header
            relist = [x.split(":")[1] for x in relist]
            return (',').join([(x + "-" + str(relist.count(x))) for x in set(relist)])
        else:
            return None

def extract_variant_ZAVariant(sample): #Change header
    res_file = os.path.join('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Annotation/user_aa', str(sample) + ".tsv")
    if os.path.exists(res_file):
        df = pd.read_csv(res_file, sep="\t")
        if 'ZAVariant' in df.columns: #Change header
            df = df.drop_duplicates(subset=['ZAVariant'], keep="first") #Change header
            df = df[df.ALT_FREQ >= 0.4]
            relist = df['ZAVariant'].dropna().tolist() #Change header
            relist = [x.split(":")[1] for x in relist]
            return (',').join([(x + "-" + str(relist.count(x))) for x in set(relist)])
        else:
            return None

def extract_variant_P2Variant(sample): #Change header
    res_file = os.path.join('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Annotation/user_aa', str(sample) + ".tsv")
    if os.path.exists(res_file):
        df = pd.read_csv(res_file, sep="\t")
        if 'P2Variant' in df.columns: #Change header
            df = df.drop_duplicates(subset=['P2Variant'], keep="first") #Change header
            df = df[df.ALT_FREQ >= 0.4]
            relist = df['P2Variant'].dropna().tolist() #Change header
            relist = [x.split(":")[1] for x in relist]
            return (',').join([(x + "-" + str(relist.count(x))) for x in set(relist)])
        else:
            return None

def extract_variant_E484K(sample): #Change header
    res_file = os.path.join('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Annotation/user_aa', str(sample) + ".tsv")
    if os.path.exists(res_file):
        df = pd.read_csv(res_file, sep="\t")
        if 'E484K' in df.columns: #Change header
            df = df.drop_duplicates(subset=['E484K'], keep="first") #Change header
            df = df[df.ALT_FREQ >= 0.4]
            relist = df['E484K'].dropna().tolist() #Change header
            relist = [x.split(":")[1] for x in relist]
            return (',').join([(x + "-" + str(relist.count(x))) for x in set(relist)])
        else:
            return None

def extract_variant_Note(sample): #Change header
    res_file = os.path.join('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Annotation/user_aa', str(sample) + ".tsv")
    if os.path.exists(res_file):
        df = pd.read_csv(res_file, sep="\t")
        if 'Variants_Note' in df.columns: #Change header
            df = df.drop_duplicates(subset=['Variants_Note'], keep="first") #Change header
            df = df[df.ALT_FREQ >= 0.4]
            relist = df['Variants_Note'].dropna().tolist() #Change header
            relist = [x.split(":")[1] for x in relist]
            return (',').join([(x + "-" + str(relist.count(x))) for x in set(relist)])
        else:
            return None
def extract_variant_M2(sample): #Change header
    res_file = os.path.join('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Annotation/user_aa', str(sample) + ".tsv")
    if os.path.exists(res_file):
        df = pd.read_csv(res_file, sep="\t")
        if 'MVariant2' in df.columns: #Change header
            df = df.drop_duplicates(subset=['MVariant2'], keep="first") #Change header
            df = df[df.ALT_FREQ >= 0.4]
            relist = df.MVariant2.dropna().tolist() #Change header
            relist = [x.split(":")[1] for x in relist]
            return (',').join([(x + "-" + str(relist.count(x))) for x in set(relist)])
        else:
            return None

def extract_variant_RBD(sample): #Change header
    res_file = os.path.join('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Annotation/user_aa', str(sample) + ".tsv")
    if os.path.exists(res_file):
        df = pd.read_csv(res_file, sep="\t")
        if 'RBD' in df.columns: #Change header
            df = df.drop_duplicates(subset=['RBD'], keep="first") #Change header
            df = df[df.ALT_FREQ >= 0.4]
            relist = df.RBD.dropna().tolist() #Change header
            relist = [x.split(":")[0] for x in relist]
            return (',').join(relist)
            # return (',').join([(x + "-" + str(relist.count(x))) for x in set(relist)])
        else:
            return None

In [106]:
final_df['UKVariant__autocolor'] = final_df.apply(lambda x: extract_variant_UK(x['id']), axis=1)
final_df['69del__autocolor'] = final_df.apply(lambda x: extract_variant_69del(x['id']), axis=1)
final_df['P1Variant__autocolor'] = final_df.apply(lambda x: extract_variant_P1Variant(x['id']), axis=1)
final_df['P2Variant__autocolor'] = final_df.apply(lambda x: extract_variant_P2Variant(x['id']), axis=1)
final_df['E484K__autocolor'] = final_df.apply(lambda x: extract_variant_E484K(x['id']), axis=1)
final_df['variant_Note__autocolor'] = final_df.apply(lambda x: extract_variant_Note(x['id']), axis=1)
final_df['MVariant2__autocolor'] = final_df.apply(lambda x: extract_variant_M2(x['id']), axis=1)
final_df['RBD__autocolor'] = final_df.apply(lambda x: extract_variant_RBD(x['id']), axis=1)
final_df['ZA__autocolor'] = final_df.apply(lambda x: extract_variant_ZAVariant(x['id']), axis=1)



In [107]:
def extract_faulty(sample):
    sample_report = os.path.join(compare_folder, "report_samples.tsv")
    df = pd.read_csv(sample_report, sep="\t")
    df['sample'] = df['sample'].astype(str)
    if str(sample) in df['sample'].values:
        return round(df[df['sample'] == str(sample)]['faulty_frac'].values[0], 3)
    else:
        return None

In [108]:
final_df['FaultyFraction__autocolor'] = final_df.apply(lambda x: extract_faulty(x['id']), axis=1)

In [109]:
#query = ['20097501','20097906','20099990','20101024','20107580','20143658','20214237','20247923','20398264','20425128','20425595','20428122','20431031','20435733','20450595','20450832','20455085','20455763','20455870','20458052','20458113','20458114','20458118','20458269','20458270','20458365','20458528','20458547','20458560','20458569','20458576','20458635','20458638','20458668','20458686','20458702','20461287','20461287-BIS','20461290','20461293','20461349','20461351','20461375','20461389','20461389-BIS','20461392','20461416','20461563','20461601','20461621','20461637','20461637-BIS','20461689','20461747','20461783','20461785','20461790','20461801','20461861','20461869','20461870','20461871','20461886','20461904','20461905','20461907','20461908','20461926','20461928','20461937','20461953','20461968','20464095','20464109','20464109-BIS','20464197','20464303','20464456','20464490','20464493','20464516','20464517','20464519','20464521','20464532','20464534','20464567','20464570','20464580','20464595','20464599','20464600','20464612','20464613','20464614','20464636']
#query_dario = [str(x) for x in query_dario]


In [110]:
#final_df['run12Maldito__autocolor'] = final_df.apply(lambda x: x['id'] in query, axis=1)


In [111]:
before_june = final_df[(final_df.month < 7) & (final_df.year == 2020)]['id'].tolist()
before_june = [str(x) for x in before_june]
final_df['before_june_2020__autocolor'] = final_df.apply(lambda x: x['id'] in before_june, axis=1)

In [112]:
final_df['id'] = final_df['id'].str.strip()
final_df = final_df.drop_duplicates(subset=['id'], keep="first")
#REORDER COLUMNS
final_df = final_df[['id','LABEL__autocolor'] + [ col for col in final_df.columns if col != 'LABEL__autocolor' and col != 'id']]

out_file = os.path.join(compare_folder, 'wgs_microreact.csv')
final_df.to_csv(out_file, index=False)

In [113]:
final_df.head()

,id,LABEL__autocolor,full_date,SEX__autocolor,AGE__autocolor,day,month,year,RUN__autocolor,DEPARTMENT__autocolor,Lineage__autocolor,Nextrain__autocolor,Resistance__autocolor,UKVariant__autocolor,69del__autocolor,P1Variant__autocolor,P2Variant__autocolor,E484K__autocolor,variant_Note__autocolor,MVariant2__autocolor,RBD__autocolor,ZA__autocolor,FaultyFraction__autocolor,before_june_2020__autocolor
0,20091284,NaN,2/4/2020,M,NaN,2,4,2020,RUN_1,NaN,B.1,20A-2,,,,,,,,MVariant2-2,,,0.001,True
1,20289362,REINFECCION,25/8/2020,F,39.694927,25,8,2020,MICRO_17,REINFECCION (95),B.1.177,20A-2;20AEU1-3,,,,,,,,MVariant2-2,,,0.000,False
2,20432945,PROFESIONAL SANITARIO,4/11/2020,F,28.247288,4,11,2020,RUN_11,TRABAJADOR (OCT_NOV),B.1.177,20A-2;20AEU1-3,,,,,,,,MVariant2-2,,,0.001,False
3,21035911,PROFESIONAL SANITARIO,23/1/2021,F,49.521111,23,1,2021,RUN_19,TRABAJADOR,B.1.177,20A-2;20AEU1-3,,,,,,,,MVariant2-2,,,0.001,False
4,21020388,DIVERSIDAD,15/01/2021,M,76.000000,15,1,2021,RUN_17,100UK,B.1.1.7,20A-2;20B-3,,UKVariant-16,6970del-1,P1Variant-2,,,B.1.525-2,MVariant2-4,"His69del,Tyr144del,Asn501Tyr,Pro681His",ZA_Variant-1,0.006,False


In [114]:
final_df[final_df.id == '21092147']

,id,LABEL__autocolor,full_date,SEX__autocolor,AGE__autocolor,day,month,year,RUN__autocolor,DEPARTMENT__autocolor,Lineage__autocolor,Nextrain__autocolor,Resistance__autocolor,UKVariant__autocolor,69del__autocolor,P1Variant__autocolor,P2Variant__autocolor,E484K__autocolor,variant_Note__autocolor,MVariant2__autocolor,RBD__autocolor,ZA__autocolor,FaultyFraction__autocolor,before_june_2020__autocolor
1260,21092147,CAM,24/2/2021,M,49.683849,24,2,2021,RUN_22,CAM (COMUNITARIO),B.1.1.7,20A-2;20B-3,,UKVariant-16,6970del-1,P1Variant-2,,,B.1.525-2,MVariant2-4,"His69del,Tyr144del,Asn501Tyr,Pro681His",ZA_Variant-1,0.001,False
